<a href="https://colab.research.google.com/github/xjdeng/MiniGPT4-image-labeler/blob/main/client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import requirements

In [ ]:
#@title
!pip install path.py==12.0.1
import base64
import requests
import time
from path import Path as path
import random
import tempfile, cv2
import pandas as pd
from tqdm import tqdm

# Set your non-streaming server URL below
Please run one of the server notebooks and follow the instructions to find this URL. Be sure to include the https:// part.

In [9]:
endpoint = "https://admissions-fees-introducing-renew.trycloudflare.com/api" #@param {type:"string"}


# Mount your Google Drive

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')

# Edit the path to the image folder below

If you didn't upload your images to the "images" folder at the top of your Google Drive, that's ok, please do the following then:
- Just click the folder icon in the left margin
- Browse down the /content/drive/MyDrive path until you find the folder with the images
- Then click the 3 dots to the right of your target folder and select "Copy Path"
- Then paste the path in the field below
- Note that the algorithm will traverse down any subfolders under that folder and search for image files to label as well.

In [11]:
imgfolder = "/content/drive/MyDrive/images" #@param {type:"string"}


# Process Images

You may need to adjust the prompt and the context if you're not getting good results but otherwise, I'd leave them alone.

Note that if you don't check the "resume" box, it'll automatically overwrite any labels.

In [ ]:
context = "\"### Human: \\nHi!\\n### Assistant: \\nHi there! How can I help you today?\\n\"" #@param {type:"string"}
prompt = "Confidently describe in 1 detailed sentence everything that you see in the photo." #@param {type:"string"}
resume = True #@param {type:"boolean"}

def label(a, prompt = prompt):
    imgtype = "data:image/jpeg;base64"
    tempdir = tempfile.TemporaryDirectory()
    img = cv2.imread(a)
    dest = "{}/{}.jpg".format(tempdir.name, str(path(a).stem))
    cv2.imwrite(dest, img)
    with open(dest, 'rb') as f:
        img_str = base64.b64encode(f.read()).decode('utf-8')
        prompt = context + f'### Human: \n {prompt} \n<img src="{imgtype},{img_str}">\n### Assistant: \n'
        return requests.post('{}/v1/generate'.format(endpoint), json={'prompt': prompt, 'stopping_strings': ['\n###'], 'max_new_tokens' : 75, \
                                                                    'temperature': 0.7, 'repetition_penalty': 1.18, 'top_p': 0.1, \
                                                                    'encoder_repetition_penalty': 1, 'top_k': 40, 'no_repeat_ngram_size': 0, \
                                                                    'typical_p': 1, 'min_length': 0, 'do_sample': True, \
                                                                    "penalty_alpha": 0, 'num_beams': 1, 'length_penalty': 1,
                                                                    'add_bos_token': True, 'skip_special_tokens': True}).json()['results'][0]['text']

queue = list(path(imgfolder).walkfiles())
random.shuffle(queue)
for f in tqdm(queue):
  namebase = f.stem
  folder = str(f.dirname())
  if f.ext == ".txt":
    continue
  txtfile = path("{}/{}.txt".format(folder, namebase))
  if resume & (txtfile.exists()):
    if txtfile.size > 0:
      continue
  try:
    txt = label(f)
    with open(txtfile,'w') as ff:
      ff.write(txt)
  except Exception as e:
    print("Warning: could not label file {} due to Exception {}".format(str(f), e))

print("Done! We've labeled all of the images that we could!")
